# Download a BioCam data set
This recording from a mouse retina was done by [Gerrit Hilgen](https://www.ncl.ac.uk/ion/staff/profile/gerrithilgen.html#background) at ION, Newcastle University, in the lab of [Evelyne Sernagor](https://www.ncl.ac.uk/ion/staff/profile/evelynesernagor.html#background). Note not the full array was recorded here (4,096 channels), but only a subset of 1056 channels, at 17kHz.

In [ ]:
import urllib.request
file_url = 'https://datasync.ed.ac.uk/public.php/webdav'

print('Downloading data file - around 8GB, so this may take a while, patience...')
password_mgr = urllib.request.HTTPPasswordMgrWithDefaultRealm()
password_mgr.add_password(None, file_url, "ovUz70bzg5pAM5Z", "HS2")
handler = urllib.request.HTTPBasicAuthHandler(password_mgr)
opener = urllib.request.build_opener(handler)
opener.open(file_url)
urllib.request.install_opener(opener)
with urllib.request.urlopen(file_url) as response, open('biocam_data.brw', 'wb') as out_file:
    data = response.read()
    out_file.write(data)
print('...done')

# Use of the `HSDetection` class

In [ ]:
import warnings
warnings.filterwarnings("ignore")

from herdingspikes import HSDetection
from herdingspikes.probe import BioCam
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
# raw data location
data_path = 'biocam_data.brw'

# detection parameters
to_localize = True
cutout_start = 10
cutout_end = 30
threshold = 16
file_name = 'ProcessedSpikes_biocam.bin'

In [ ]:
Probe = BioCam(data_path)
H = HSDetection(Probe, to_localize, cutout_start, cutout_end, threshold,
                maa=0, maxsl=12, minsl=3, ahpthr=0, out_file_name=file_name)

In [ ]:
Probe.show(figwidth=10)

In [ ]:
H.DetectFromRaw()

In [ ]:
H.LoadDetected()

In [ ]:
plt.figure(figsize=(10, 10))
H.PlotTracesChannels(1000, window_size=100)

In [ ]:
plt.figure(figsize=(8, 8))
ax = plt.subplot(111)
H.PlotAll(invert=True, s=1, alpha=0.4, ax=ax)
plt.axis('equal')

# Use of the `HSClustering` class

In [ ]:
from hs2 import HSClustering

In [ ]:
# Load from file
#C = Clustering(['ProcessedSpikes_biocam.bin', 'ProcessedSpikes_biocam.bin'], cutout_length=41)

# Or if the spikes are already in memory
# simply load from the Detection class
C = HSClustering(H)

In [ ]:
%%time
C.ShapePCA(pca_ncomponents=2, pca_whiten=True)

In [ ]:
%%time
C.CombinedClustering(alpha=0.3, bandwidth=0.3, bin_seeding=False, n_jobs=-1)

In [ ]:
plt.figure(figsize=(8, 8))
ax = plt.subplot(111)
C.PlotAll(invert=True, s=1, ax=ax, max_show=100000,show_labels=False)
plt.title("MeanShift, no bin seeding");
plt.axis('equal')

In [ ]:
units = range(16)
C.PlotShapes(units)

In [ ]:
C.PlotNeighbourhood(13,radius=0.4, alpha=0.9);

In [ ]:
C.SaveHDF5(file_name+"_sorted.hdf5")